In [285]:
:dep caw = { path = "/home/s/src/caw/caw", features = ["live", "widgets"] }

In [287]:
use caw::prelude::*;
let out: LiveStereoOut = 
    live_stereo_viz_udp(Default::default())
        .with_volume(knob("volume").build());

In [307]:
let tempo_hz: SV<f32> = sv(8.);

In [309]:
let clock: SV<bool> = sv(periodic_trig_hz(tempo_hz.clone()).build());

In [315]:
let clock_mel: SV<bool> = sv(clock.clone().divide(8));
let melody: SV<f32> = sv({
    use note::*;
    value_sequencer(clock_mel.clone(), [
        C3,
        C3,
        D3,
        C3,
        A3,
        B3,
        C3,
    ]).freq_hz()    
});

In [325]:
let clock_mel2: SV<bool> = sv(clock.clone().divide(32));

In [335]:
melody.set({
    use note::*;
    value_sequencer(clock_mel.clone(), [
        C3,
        C3,
        D3,
        C3,
        A3,
        B3,
        C4,
    ]).freq_hz()  * value_sequencer(clock_mel2.clone(), [1., 1.5, 2., 1.7]) * 0.5
});

In [340]:
let clock_snare: SV<bool> = sv(clock.clone().divide(8));

let clock_kick: SV<bool> = sv(clock.clone().divide_with_offset(8, 4));

In [385]:
let clock_kick2: SV<bool> = sv(clock.clone().divide_with_offset(16, 7));

In [363]:
chord_progression.set({
    use note_name::*;
    use chord::*;
    value_sequencer(clock_mel2.clone(), [
        chord(C, MAJOR.infer_7()),
        chord(F, MINOR.infer_7()),
        chord(A, MINOR.infer_7()),
        chord(D, MAJOR.infer_7()),
    ])
});

In [362]:
arp_notes.set({
    chord_progression.clone()
        .map(Some)
        .key_events(ChordVoiceConfig::default().with_inversion(Inversion::InOctave { octave_base: note::C3 }))
        .arp(clock.clone(), ArpConfig::default().with_extend_octaves_high(2).with_shape(ArpShape::UpDown))
});

In [364]:
tempo_hz.set(4.);

In [396]:
out.set_channel(|channel| {
    let melody = {
        arp_notes.clone().poly_voices(12).into_iter().map(|voice| {
            let lfo1 = oscillator(Sine, 0.1).build().signed_to_01() * 5000.;
            let rand_cutoff = noise::white().filter(sample_and_hold(clock.clone())).signed_to_01() * 5000.;
            let lpf_env = adsr(voice.key_press_trig).r(0.2).build().exp_01(1.);
            super_saw(voice.note.freq_hz()).num_oscillators(16).build()
                .filter(low_pass::default((500. + rand_cutoff + lfo1) * lpf_cutoff.clone() * lpf_env)
                        .q(lpf_resonance.clone() * 2.))
                .filter(chorus())
                
        }).sum::<Sig<_>>()
        .filter(reverb())
        .filter(delay_trig(clock.clone()))
        .filter(high_pass::default(1.))
    };
    let bass = {
        let rand_cutoff = noise::white().filter(sample_and_hold(clock.clone())).signed_to_01() * 5000.;
        let osc = oscillator(
            Saw,
            chord_progression.clone().map(|chord| chord.root.in_octave(OCTAVE_2)).freq_hz(),
        ).build();
        let env = adsr(clock_mel2.clone().trig_to_gate(4.)).a(1.).r(4.).build();
        osc.filter(low_pass::default((200. + rand_cutoff) * env * knob("bye").build()).q(1.))
            .filter(chorus()
                    .num_voices(2).delay_s(0.01).depth_s(0.01).lfo_rate_hz(0.1)
                    .lfo_offset(ChorusLfoOffset::Interleave(channel))
                   )
            //.filter(down_sample(5.))
            //.filter(quantizer(10.))
            .filter(compressor().threshold(0.5).scale(5.).ratio(0.2))
    } * bass_vol.clone();
    let drums = {
        //clock_kick.clone().trig(drum::kick()) +
        //clock_kick2.clone().trig(drum::kick()) +
        clock_snare.clone().trig(drum::snare()).filter(compressor().threshold(0.5).scale(5.))
    } * drums_vol.clone();
     bass
});


thread 'main' panicked at src/main.rs:77:22:
called `Result::unwrap()` on an `Err` value: Connection refused (os error 111)

Stack backtrace:
   0: anyhow::error::<impl core::convert::From<E> for anyhow::Error>::from
   1: caw_midi_udp_client::MidiUdpClient::send
   2: caw_midi_udp_widgets_app::main
   3: std::sys::backtrace::__rust_begin_short_backtrace
   4: std::rt::lang_start::{{closure}}
   5: std::rt::lang_start_internal
   6: main
   7: __libc_start_call_main
   8: __libc_start_main_impl
   9: _start
stack backtrace:

thread 'main' panicked at src/main.rs: 77: 22 :
0called `Result::unwrap()` on an `Err` value: Connection refused (os error 111)

Stack backtrace:
   0: anyhow::error::<impl core::convert::From<E> for anyhow::Error>::from
   1: caw_midi_udp_client::MidiUdpClient::send
   2: caw_midi_udp_widgets_app::main
   3: std::sys::backtrace::__rust_begin_short_backtrace
   4: std::rt::lang_start::{{closure}}
   5: std::rt::lang_start_internal
   6: main
   7: __libc_start_cal

In [301]:
let lpf_cutoff: SV<f32> = sv(knob("lpf_cutoff").build());

In [304]:
let lpf_resonance: SV<f32> = sv(knob("lpf_res").build());

In [328]:
let drums_vol: SV<f32> = sv(knob("drums vol").build());

In [375]:
let bass_vol: SV<f32> = sv(knob("bass vol").build());